In [8]:
import pandas as pd

df6 = pd.read_csv('train.csv', encoding='UTF-8', sep=',')

print(df6)

            ID Customer_ID     Month           Name   Age          SSN  \
0       0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265   
1       0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265   
2       0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265   
3       0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265   
4       0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265   
...        ...         ...       ...            ...   ...          ...   
99995  0x25fe9  CUS_0x942c     April          Nicks    25  078-73-5990   
99996  0x25fea  CUS_0x942c       May          Nicks    25  078-73-5990   
99997  0x25feb  CUS_0x942c      June          Nicks    25  078-73-5990   
99998  0x25fec  CUS_0x942c      July          Nicks    25  078-73-5990   
99999  0x25fed  CUS_0x942c    August          Nicks    25  078-73-5990   

      Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  \
0      Scientist      19114.12

C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\3670776720.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df6 = pd.read_csv('train.csv', encoding='UTF-8', sep=',')


In [9]:
#Preprocesamiento y limpieza inicial dataset train

#Estadísticas descriptivas iniciales
print(df6.describe())

#Ver tipos de datos
print('Tipos de datos:')
print(df6.dtypes)

       Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  \
count           84998.000000      100000.000000     100000.00000   
mean             4194.170850          17.091280         22.47443   
std              3183.686167         117.404834        129.05741   
min               303.645417          -1.000000          0.00000   
25%              1625.568229           3.000000          4.00000   
50%              3093.745000           6.000000          5.00000   
75%              5957.448333           7.000000          7.00000   
max             15204.633333        1798.000000       1499.00000   

       Interest_Rate  Delay_from_due_date  Num_Credit_Inquiries  \
count  100000.000000        100000.000000          98035.000000   
mean       72.466040            21.068780             27.754251   
std       466.422621            14.860104            193.177339   
min         1.000000            -5.000000              0.000000   
25%         8.000000            10.000000           

In [10]:
#Identificamos caracteres extraños en cada columna

import re

# Función para identificar caracteres extraños en cada columna
def detectar_caracteres_extranos(df6):
    caracteres_por_columna = {}
    for col in df6.columns:
        valores_unicos = ''.join(df6[col].astype(str).unique())  # Unir todos los valores únicos como string
        caracteres_unicos = set(re.findall(r'[^a-zA-Z0-9\s.,-]', valores_unicos))  # Buscar caracteres no alfanuméricos comunes
        if caracteres_unicos:  # Solo guardar columnas con caracteres extraños
            caracteres_por_columna[col] = caracteres_unicos
    return caracteres_por_columna

# Identificar caracteres extraños en el dataframe
caracteres_extranos = detectar_caracteres_extranos(df6)

# Mostrar los resultados
for columna, caracteres in caracteres_extranos.items():
    print(f"Columna: {columna} -> Caracteres extraños detectados: {caracteres}")

Columna: Customer_ID -> Caracteres extraños detectados: {'_'}
Columna: Name -> Caracteres extraños detectados: {'"'}
Columna: Age -> Caracteres extraños detectados: {'_'}
Columna: SSN -> Caracteres extraños detectados: {'$', '&', '*', '@', '#', '%'}
Columna: Occupation -> Caracteres extraños detectados: {'_'}
Columna: Annual_Income -> Caracteres extraños detectados: {'_'}
Columna: Num_of_Loan -> Caracteres extraños detectados: {'_'}
Columna: Num_of_Delayed_Payment -> Caracteres extraños detectados: {'_'}
Columna: Changed_Credit_Limit -> Caracteres extraños detectados: {'_'}
Columna: Credit_Mix -> Caracteres extraños detectados: {'_'}
Columna: Outstanding_Debt -> Caracteres extraños detectados: {'_'}
Columna: Amount_invested_monthly -> Caracteres extraños detectados: {'_'}
Columna: Payment_Behaviour -> Caracteres extraños detectados: {'_', '@', '!', '#', '%'}
Columna: Monthly_Balance -> Caracteres extraños detectados: {'_'}


In [11]:
import numpy as np

#Aplicar limpieza caracteres extraños
#Función para limpiar el dataset según las reglas definidas
def limpiar_columnas(df6):
    #Eliminar guiones bajos y convertir a numérico en columnas específicas
    df6['Age'] = df6['Age'].str.replace('_','', regex=True).astype(float)
    df6['Annual_Income'] = df6['Annual_Income'].str.replace('_','',regex=True).astype(float)

    #Reemplazar guiones bajos por espacios en columnas específicas
    columnas_reemplazo_espacio = ['Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit','Credit_Mix',]
    for col in columnas_reemplazo_espacio:
        df6[col] = df6[col].str.replace('_',' ', regex=True)

    #Eliminar registros con caracteres inválidos
    df6 = df6[~df6['Payment_Behaviour'].str.contains(r'[!%@#]', regex=True)]

    #Reemplazar guines bajos por espacios en Payment_Behaviour
    df6['Payment_Behaviour'] = df6['Payment_Behaviour'].str.replace('_',' ', regex=True)

    return df6

#Aplicar limpieza al dataframe
df6_clean = limpiar_columnas(df6)

#Hacer conversiones adicionales
#Lista de columnas que deben ser numéricas
columnas_a_convertir = ['Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit']

#Reemplazar espacios vacíos por NaN
df6_clean[columnas_a_convertir] = df6_clean[columnas_a_convertir].replace(' ', np.nan)

#Convertir a numérico
df6_clean[columnas_a_convertir] = df6_clean[columnas_a_convertir].astype(float)

#Verificar si aún hay NaN
print('Verificar si aún hay Nan en columnas a convertir')
print(df6_clean[columnas_a_convertir].isnull().sum())

#Revisar si hay columnas que deberían ser numéricas
num_cols = ['Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Outstanding_Debt', 'Num_of_Loan',
            'Num_of_Delayed_Payment','Changed_Credit_Limit',
            'Credit_Utilization_Ratio','Credit_History_Age','Total_EMI_per_month',
            'Amount_invested_monthly','Monthly_Balance']
for col in num_cols:
    df6[col] = pd.to_numeric(df6[col], errors='coerce')

#Verificar cambios
print(df6.dtypes)

C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\323143973.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Payment_Behaviour'] = df6['Payment_Behaviour'].str.replace('_',' ', regex=True)
C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\323143973.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6_clean[columnas_a_convertir] = df6_clean[columnas_a_convertir].replace(' ', np.nan)
C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\323143973.py:34: SettingWithCopyWarning: 
A value is trying to be s

Verificar si aún hay Nan en columnas a convertir
Num_of_Loan                  0
Num_of_Delayed_Payment    6420
Changed_Credit_Limit      1942
dtype: int64
ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                         float64
SSN                          object
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                   int64
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age          float64
Payment_of_Min_Amount        object
Total_EMI_per_mon

In [12]:
#Diagnóstico de valores nulos
for i in df6_clean.columns:
    print(i+':'+ str(df6_clean[i].isnull().sum()))

print(df6_clean)

ID:0
Customer_ID:0
Month:0
Name:9240
Age:0
SSN:0
Occupation:0
Annual_Income:0
Monthly_Inhand_Salary:13842
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Type_of_Loan:10520
Delay_from_due_date:0
Num_of_Delayed_Payment:6420
Changed_Credit_Limit:1942
Num_Credit_Inquiries:1806
Credit_Mix:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:8341
Payment_of_Min_Amount:0
Total_EMI_per_month:0
Amount_invested_monthly:4132
Payment_Behaviour:0
Monthly_Balance:1105
Credit_Score:0
            ID Customer_ID     Month           Name    Age          SSN  \
0       0x1602   CUS_0xd40   January  Aaron Maashoh   23.0  821-00-0265   
1       0x1603   CUS_0xd40  February  Aaron Maashoh   23.0  821-00-0265   
2       0x1604   CUS_0xd40     March  Aaron Maashoh -500.0  821-00-0265   
3       0x1605   CUS_0xd40     April  Aaron Maashoh   23.0  821-00-0265   
4       0x1606   CUS_0xd40       May  Aaron Maashoh   23.0  821-00-0265   
...        ...         ...       ...     

In [13]:
# Para manejar valores nulos, nos apoyaremos en el registro duplicado correspondiente (considerando que son datos de panel)

# Función de imputación por duplicados
def imputar_con_logica_duplicados(df6_clean, columnas_a_imputar):
    df6_clean = df6_clean.copy()

    #Ordenamos por customer_ID y Month para mantener coherencia temporal
    df6_clean = df6_clean.sort_values(by=['Customer_ID','Month']).reset_index(drop=True)

    for col in columnas_a_imputar:
        #Trabajamos por grupos de clientes
        def imputar_grupo(grupo):
            #Si todo está NaN en esa columna para ese cliente, lo dejamos tal cual
            if grupo[col].isnull().all():
                return grupo[col]
            
            #Aplicamos imputación con duplicados
            valores = grupo[col].copy()

            for idx in range(len(valores)):
                if pd.isna(valores.iloc[idx]):
                    if idx == 0: #Primer registro del cliente
                        siguiente_valido = valores.iloc[idx+1:].dropna()
                        if not siguiente_valido.empty:
                            valores.iloc[idx] = siguiente_valido.iloc[0]
                    elif idx == len(valores) - 1: #Ultimo registro
                        anterior_valido = valores.iloc[:idx].dropna()
                        if not anterior_valido.empty:
                            valores.iloc[idx] = anterior_valido.iloc[-1]
                    else: #Registro intermedio
                        siguiente_valido = valores.iloc[idx+1:].dropna()
                        anterior_valido = valores.iloc[:idx].dropna()
                        if not siguiente_valido.empty:
                            valores.iloc[idx] = siguiente_valido.iloc[0]
                        elif not anterior_valido.empty:
                            valores.iloc[idx] = anterior_valido.iloc[-1]
            return valores
        
        #Aplicamos la función a cada grupo de clientes
        df6_clean[col] = df6_clean.groupby('Customer_ID').apply(imputar_grupo).reset_index(level=0, drop=True)

    return df6_clean

# Aplicamos limpiza invocando la función a las columnas con valores NaN (del diagnóstico)
columnas_con_nan = [
    'Name','Monthly_Inhand_Salary', 'Type_of_Loan', 'Num_of_Delayed_Payment',
    'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_History_Age',
    'Amount_invested_monthly', 'Monthly_Balance'
]

# Aplicar la función
df6_clean = imputar_con_logica_duplicados(df6_clean, columnas_con_nan)
   
print(df6_clean)

# Diagnosticamos nulos nuevamente
for i in df6_clean.columns:
    print(i+':'+ str(df6_clean[i].isnull().sum()))

print('\n Como se puede observar en el último diagnóstico de valores nulos, la mayoría de columnas fueron imputadas utilizando' \
' registros duplicados mediante backfill ó ffill según correspondía. Sin embargo, nos queda una columna importante que servirá como' \
' descriptor para los modelos de Clasificación, siendo esta "Type_of_Loan", una variable de tipo object que describe los tipos de ' \
'deuda que tiene el cliente. Dado que no se tiene referencia usando su propio Customer_ID, se rellenará la información con la palabra "Unknown",'
' la cual referenciará la falta de información, y que los modelos de Clasificación interiorizarán para manejar sus predicciones con información faltante.' \
'Sin embargo, antes es necesario comprobar el estado actual del dataframe para reducir al máximo los NaNs posibles en esta columna.')

C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\1379881226.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df6_clean[col] = df6_clean.groupby('Customer_ID').apply(imputar_grupo).reset_index(level=0, drop=True)
C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\1379881226.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df6_clean[col] = df6_clean.groupby('Customer_ID').apply(imputar_grupo).reset_inde

            ID Customer_ID     Month            Name   Age          SSN  \
0      0x16291  CUS_0x1000    August  Alistair Barrf  18.0  913-74-1218   
1      0x1628b  CUS_0x1000  February  Alistair Barrf  17.0  913-74-1218   
2      0x1628a  CUS_0x1000   January  Alistair Barrf  17.0  913-74-1218   
3      0x16290  CUS_0x1000      July  Alistair Barrf  18.0  913-74-1218   
4      0x1628f  CUS_0x1000      June  Alistair Barrf  18.0  913-74-1218   
...        ...         ...       ...             ...   ...          ...   
92395  0x25afa   CUS_0xffd   January        Damouniq  29.0  832-88-8320   
92396  0x25b00   CUS_0xffd      July        Damouniq  30.0  832-88-8320   
92397  0x25aff   CUS_0xffd      June        Damouniq  29.0  832-88-8320   
92398  0x25afc   CUS_0xffd     March        Damouniq  29.0  832-88-8320   
92399  0x25afe   CUS_0xffd       May        Damouniq  29.0  832-88-8320   

      Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0         Lawyer      

In [14]:
# Convertir la columna Credit_History_Age a años (de object a float)
def convertir_credit_history_a_anios(texto):
    import numpy as np
    if pd.isnull(texto):
        return np.nan
    try:
        partes = texto.lower().replace("and", "").replace("months","month").replace("years","year").split()
        anios = int(partes[0]) if 'year' in partes[1] else 0
        meses = int(partes[2]) if len(partes) > 2 and 'month' in partes[3] else 0
        return round(anios + meses /12, 2)
    except:
        return np.nan
    
df6_clean['Credit_History_Age'] = df6_clean['Credit_History_Age'].apply(convertir_credit_history_a_anios)

#Identificamos caracteres extraños en cada columna luego de la imputación inicial

import re

# Función para identificar caracteres extraños en cada columna
def detectar_caracteres_extranos(df6):
    caracteres_por_columna = {}
    for col in df6.columns:
        valores_unicos = ''.join(df6[col].astype(str).unique())  # Unir todos los valores únicos como string
        caracteres_unicos = set(re.findall(r'[^a-zA-Z0-9\s.,-]', valores_unicos))  # Buscar caracteres no alfanuméricos comunes
        if caracteres_unicos:  # Solo guardar columnas con caracteres extraños
            caracteres_por_columna[col] = caracteres_unicos
    return caracteres_por_columna

# Identificar caracteres extraños en el dataframe
caracteres_extranos = detectar_caracteres_extranos(df6)

# Mostrar los resultados
for columna, caracteres in caracteres_extranos.items():
    print(f"Columna: {columna} -> Caracteres extraños detectados: {caracteres}")

print('\n Podemos observar que luego de nuestra imputación de NaNs mediante la lógica de duplicados, aún quedan caracteres extraños en descriptores ' \
'esenciales para entrenar modelos de Clasificación, como Occupation y Payment_Behaviour. Asimismo, si revisamos a profundidad nuestro dataframe hasta ' \
'ahora, existen valores incoherentes y/o sucios, como Age -500 o Num_of_Loan 1094, los cuales entorpecen tanto el procesamiento como el entrenamiento. Para ello, se aplicará ' \
'limpieza usando la misma lógica de duplicados, pero esta vez con criterios de imputación basados en la moda de los duplicados.')

Columna: Customer_ID -> Caracteres extraños detectados: {'_'}
Columna: Name -> Caracteres extraños detectados: {'"'}
Columna: SSN -> Caracteres extraños detectados: {'$', '&', '*', '@', '#', '%'}
Columna: Occupation -> Caracteres extraños detectados: {'_'}
Columna: Payment_Behaviour -> Caracteres extraños detectados: {'_', '@', '!', '#', '%'}

 Podemos observar que luego de nuestra imputación de NaNs mediante la lógica de duplicados, aún quedan caracteres extraños en descriptores esenciales para entrenar modelos de Clasificación, como Occupation y Payment_Behaviour. Asimismo, si revisamos a profundidad nuestro dataframe hasta ahora, existen valores incoherentes y/o sucios, como Age -500 o Num_of_Loan 1094, los cuales entorpecen tanto el procesamiento como el entrenamiento. Para ello, se aplicará limpieza usando la misma lógica de duplicados, pero esta vez con criterios de imputación basados en la moda de los duplicados.


In [15]:
# Función de limpieza utilizando duplicados mediante Name

import pandas as pd
import numpy as np
import re

def limpieza_avanzada_por_duplicados(df, columna_name='Name'):
    df = df.copy()

    # Convertimos a numérico las columnas posibles para detección correcta
    for col in df.select_dtypes(include='object'):
        try:
            df[col] = pd.to_numeric(df[col], errors='ignore')
        except:
            pass

    # 1. Trabajar primero con columnas numéricas
    columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns

    for col in columnas_numericas:
        if col == columna_name:
            continue  # No modificar el nombre

        # Calcular la moda por el nombre del cliente
        moda_por_cliente = df.groupby(columna_name)[col].agg(
            lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
        )

        # Imputar donde hay NaN o donde el valor difiere de la moda
        for idx, row in df.iterrows():
            customer = row[columna_name]
            valor = row[col]
            moda = moda_por_cliente.get(customer, np.nan)

            if pd.isna(valor) or (not pd.isna(moda) and valor != moda):
                df.at[idx, col] = moda

        print(f"[DEBUG] Imputación numérica aplicada en '{col}'")

    # 2. Ahora columnas categóricas con caracteres extraños o NaN
    columnas_categoricas = df.select_dtypes(include='object').columns

    patron_extrano = re.compile(r'[^a-zA-Z0-9\s.,-]')

    for col in columnas_categoricas:
        if col == columna_name:
            continue

        def es_valor_extrano(x):
            if pd.isna(x): return True
            return bool(patron_extrano.search(str(x)))

        mascara_extranos = df[col].apply(es_valor_extrano)

        # Obtener moda categórica por nombre del cliente (para valores válidos)
        moda_cat_por_cliente = (
            df[~mascara_extranos].groupby(columna_name)[col]
            .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        )

        # Aplicar la imputación
        for idx in df[mascara_extranos].index:
            customer = df.at[idx, columna_name]
            moda = moda_cat_por_cliente.get(customer, np.nan)
            if pd.notna(moda):
                df.at[idx, col] = moda

        print(f"[DEBUG] Imputación categórica aplicada en '{col}'")

    return df

#Ejecutar función
df6_clean = limpieza_avanzada_por_duplicados(df6_clean)

C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\2307353719.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')


[DEBUG] Imputación numérica aplicada en 'Age'
[DEBUG] Imputación numérica aplicada en 'Annual_Income'
[DEBUG] Imputación numérica aplicada en 'Monthly_Inhand_Salary'
[DEBUG] Imputación numérica aplicada en 'Num_Bank_Accounts'
[DEBUG] Imputación numérica aplicada en 'Num_Credit_Card'
[DEBUG] Imputación numérica aplicada en 'Interest_Rate'
[DEBUG] Imputación numérica aplicada en 'Num_of_Loan'
[DEBUG] Imputación numérica aplicada en 'Delay_from_due_date'
[DEBUG] Imputación numérica aplicada en 'Num_of_Delayed_Payment'
[DEBUG] Imputación numérica aplicada en 'Changed_Credit_Limit'
[DEBUG] Imputación numérica aplicada en 'Num_Credit_Inquiries'
[DEBUG] Imputación numérica aplicada en 'Credit_Utilization_Ratio'
[DEBUG] Imputación numérica aplicada en 'Credit_History_Age'
[DEBUG] Imputación numérica aplicada en 'Total_EMI_per_month'
[DEBUG] Imputación categórica aplicada en 'ID'
[DEBUG] Imputación categórica aplicada en 'Customer_ID'
[DEBUG] Imputación categórica aplicada en 'Month'
[DEBUG] Im

C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\2307353719.py:58: UserWarning: Unable to sort modes: '<' not supported between instances of 'float' and 'str'
  .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
C:\Users\esteb\AppData\Local\Temp\ipykernel_4276\2307353719.py:58: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)


[DEBUG] Imputación categórica aplicada en 'Monthly_Balance'
[DEBUG] Imputación categórica aplicada en 'Credit_Score'


In [16]:
# Separar columna 'Type_of_Loan' en categorías individuales para mejorar codificación y granularidad del dataframe
import pandas as pd
import numpy as np

# Crear una copia temporal del dataframe
df6_temp = df6_clean.copy()

# Limpiar comillas dobles, espacios y reemplazar "and" por ","
df6_temp['Type_of_Loan'] = (
    df6_temp['Type_of_Loan']
    .str.replace('"','',regex=False)
    .str.replace(' and ',', ', regex=False)
    .str.strip()
)

# Extraer todos los tipos únicos de prestamos (ignorando los NaN)
tipos_prestamo = set()
df6_temp['Type_of_Loan'].dropna().apply(lambda x: tipos_prestamo.update([i.strip() for i in x.split(',') if i.strip() != '']))

# Crear una columna por cada tipo de préstamo
for tipo in tipos_prestamo:
    df6_temp[tipo] = df6_temp['Type_of_Loan'].apply(
        lambda x: 'Yes' if pd.notna(x) and tipo in x else ('No' if pd.notna(x) else np.nan)
    )

# Eliminar la columna original
df6_temp.drop(columns=['Type_of_Loan'], inplace=True)

print(df6_temp)

# Diagnóstico
# Diagnóstico
print(f"Tipos de préstamo extraídos: {sorted(tipos_prestamo)}")
print("Valores únicos por nueva columna (debería ser 'Yes', 'No' o NaN):\n")
print(df6_temp[list(tipos_prestamo)].nunique())

# Reasignamos al dataframe original

df6_clean = df6_temp.copy()

            ID Customer_ID     Month            Name   Age          SSN  \
0      0x16291  CUS_0x1000    August  Alistair Barrf  45.0  913-74-1218   
1      0x1628b  CUS_0x1000  February  Alistair Barrf  45.0  913-74-1218   
2      0x1628a  CUS_0x1000   January  Alistair Barrf  45.0  913-74-1218   
3      0x16290  CUS_0x1000      July  Alistair Barrf  45.0  913-74-1218   
4      0x1628f  CUS_0x1000      June  Alistair Barrf  45.0  913-74-1218   
...        ...         ...       ...             ...   ...          ...   
92395  0x25afa   CUS_0xffd   January        Damouniq  29.0  832-88-8320   
92396  0x25b00   CUS_0xffd      July        Damouniq  29.0  832-88-8320   
92397  0x25aff   CUS_0xffd      June        Damouniq  29.0  832-88-8320   
92398  0x25afc   CUS_0xffd     March        Damouniq  29.0  832-88-8320   
92399  0x25afe   CUS_0xffd       May        Damouniq  29.0  832-88-8320   

      Occupation  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0         Lawyer      

In [17]:
# Verificar detalles finales de limpieza 

print("\n[INFO] Ocupaciones únicas antes de limpiar:")
print(sorted(df6_clean['Occupation'].dropna().unique()))

print('\n[INFO] Observamos que existen registros con el valor sucio "_______". Se inicia proceso de imputación por duplicado...')

# Creamos una copia para no afectar el original
df6_temp = df6_clean.copy()

# Creamos la máscara inicial
mascara_sucia = df6_temp['Occupation'] == '_______'
sucios_previos = mascara_sucia.sum()

# Bucle con bandera de control
while True:
    # Calcular la moda por Customer_ID (excluyendo registros sucios)
    moda_occupation_por_cliente = (
        df6_temp[~mascara_sucia]
        .groupby('Customer_ID')['Occupation']
        .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    )

    # Imputar donde sea posible
    for idx in df6_temp[mascara_sucia].index:
        customer = df6_temp.at[idx, 'Customer_ID']
        ocupacion_moda = moda_occupation_por_cliente.get(customer, np.nan)
        if pd.notna(ocupacion_moda):
            df6_temp.at[idx, 'Occupation'] = ocupacion_moda

    # Recalcular la máscara y comparar
    mascara_sucia = df6_temp['Occupation'] == '_______'
    sucios_actuales = mascara_sucia.sum()
    print(f"[DEBUG] Registros aún sucios: {sucios_actuales}")

    # Verificar si no hubo progreso → cortar
    if sucios_actuales == sucios_previos:
        print("[INFO] No se puede imputar más. Se procede a reemplazar por 'Unknown'.")
        break

    sucios_previos = sucios_actuales

# Reemplazar los que no pudieron imputarse
df6_temp.loc[df6_temp['Occupation'] == '_______', 'Occupation'] = 'Unknown'

# Asignar al dataframe limpio final
df6_clean = df6_temp.copy()

# Verificación final
print("\n[INFO] Ocupaciones únicas después de limpieza:")
print(sorted(df6_clean['Occupation'].dropna().unique()))


[INFO] Ocupaciones únicas antes de limpiar:
['Accountant', 'Architect', 'Developer', 'Doctor', 'Engineer', 'Entrepreneur', 'Journalist', 'Lawyer', 'Manager', 'Mechanic', 'Media_Manager', 'Musician', 'Scientist', 'Teacher', 'Writer', '_______']

[INFO] Observamos que existen registros con el valor sucio "_______". Se inicia proceso de imputación por duplicado...
[DEBUG] Registros aún sucios: 0
[DEBUG] Registros aún sucios: 0
[INFO] No se puede imputar más. Se procede a reemplazar por 'Unknown'.

[INFO] Ocupaciones únicas después de limpieza:
['Accountant', 'Architect', 'Developer', 'Doctor', 'Engineer', 'Entrepreneur', 'Journalist', 'Lawyer', 'Manager', 'Mechanic', 'Media_Manager', 'Musician', 'Scientist', 'Teacher', 'Writer']


In [18]:
# Diagnosticamos nulos nuevamente
for i in df6_clean.columns:
    print(i+':'+ str(df6_clean[i].isnull().sum()))

# Etiquetar como 'Unknown' valores NaN de la columnas individualizadas
df6_clean[list(tipos_prestamo)] = df6_clean[list(tipos_prestamo)].fillna('Unknown')

# Etiquetar como 'Unknown' valores NaN de la columna Name
df6_clean['Name'] = df6_clean['Name'].fillna('Unknown')

# Etiquetar como 'Unknown' valores vacíos de la columna Credit_Mix
df6_clean['Credit_Mix'] = df6_clean['Credit_Mix'].replace(' ','Unknown')

#Diagnosticamos por última vez los valores nulos en las columnas del dataframe
for i in df6_clean.columns:
    print(i+':'+ str(df6_clean[i].isnull().sum()))

#Verificar tamaño del dataframe luego de la limpieza
print(df6_clean.shape)

ID:0
Customer_ID:0
Month:0
Name:5
Age:0
SSN:0
Occupation:0
Annual_Income:0
Monthly_Inhand_Salary:0
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Delay_from_due_date:0
Num_of_Delayed_Payment:0
Changed_Credit_Limit:0
Num_Credit_Inquiries:0
Credit_Mix:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:0
Payment_of_Min_Amount:0
Total_EMI_per_month:0
Amount_invested_monthly:0
Payment_Behaviour:0
Monthly_Balance:0
Credit_Score:0
Payday Loan:7300
Auto Loan:7300
Credit-Builder Loan:7300
Not Specified:7300
Debt Consolidation Loan:7300
Student Loan:7300
Mortgage Loan:7300
Personal Loan:7300
Home Equity Loan:7300
ID:0
Customer_ID:0
Month:0
Name:0
Age:0
SSN:0
Occupation:0
Annual_Income:0
Monthly_Inhand_Salary:0
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Delay_from_due_date:0
Num_of_Delayed_Payment:0
Changed_Credit_Limit:0
Num_Credit_Inquiries:0
Credit_Mix:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:0
Payment_of_M

In [19]:
# Transformación de columnas df6_clean final

# Columnas que deben ser numéricas pero están como object
columnas_a_convertir = ['Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance']

# Convertir a numérico (float64), forzando errores a NaN
for col in columnas_a_convertir:
    df6_clean[col] = pd.to_numeric(df6_clean[col], errors='coerce')

# Verificación de tipos
print(df6_clean[columnas_a_convertir].dtypes)

Outstanding_Debt           float64
Amount_invested_monthly    float64
Monthly_Balance            float64
dtype: object


In [20]:
df6_clean.to_csv('train_clean(sin_codificar).csv', index=False)

import os
print(os.getcwd())

d:\Ciencia de Datos y Análisis con Python\Bases de Datos Kaggle\Credit Score Classification feb\Archivos Repositorio Github


In [21]:
#Codificación de variables categóricas dataframe de entrenamiento

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Copia del dataframe limpio
df6_cleaned_final = df6_clean.copy()

# Limpieza de 'Occupation'

# Lista completa de ocupaciones válidas
ocupaciones_validas = [
    'Lawyer', 'Mechanic', 'Media_Manager', 'Doctor', 'Journalist',
    'Accountant', 'Manager', 'Entrepreneur', 'Scientist', 'Architect',
    'Teacher', 'Engineer', 'Writer', 'Developer', 'Musician', 'Unknown'
]

# Reemplazar cualquier valor no listado por 'Other'
df6_cleaned_final['Occupation'] = df6_cleaned_final['Occupation'].apply(
    lambda x: x if x in ocupaciones_validas else 'Other'
)

# Definimos un mapeo numérico del 1 al 17 (sin significado ordinal) para los valores de Occupation
occupation_categories = ocupaciones_validas + ['Other']
occupation_mapping = {name: idx + 1 for idx, name in enumerate(occupation_categories)}
df6_cleaned_final['Occupation_Encoded'] = df6_cleaned_final['Occupation'].map(occupation_mapping)

#Verificación
print('Verificación codificación Occupation')
print(df6_cleaned_final[['Occupation', 'Occupation_Encoded']].drop_duplicates().sort_values('Occupation_Encoded'))

# Reemplazar columna original por codificada (Occupation_Encoded)
df6_cleaned_final['Occupation'] = df6_cleaned_final['Occupation_Encoded']
df6_cleaned_final.drop(columns=['Occupation'], inplace=True)

# Codificación de ordinales

# Credit_Mix (ordinal: Bad < Standard < Good, Unknown)
mix_mapping = {'Bad': 0, 'Standard': 1, 'Good': 2, 'Unknown':3}
df6_cleaned_final['Credit_Mix_Encoded'] = df6_cleaned_final['Credit_Mix'].map(mix_mapping)

# Payment_of_Min_Amount (ordinal: NM < No < Yes)
paymin_mapping = {'NM': 0, 'No': 1, 'Yes': 2}
df6_cleaned_final['Payment_of_Min_Amount_Encoded'] = df6_cleaned_final['Payment_of_Min_Amount'].map(paymin_mapping)

# Eliminamos las columnas originales de Credit_Mix y Payment_of_Min_Amount 
df6_cleaned_final.drop(columns=['Credit_Mix', 'Payment_of_Min_Amount'], inplace=True)

# Definir mapeo de categorías de Payment_Behaviour
payment_behaviour_mapping = {
    'High spent Medium value payments': 1,
    'High spent Small value payments': 2,
    'Low spent Large value payments': 3,
    'Low spent Small value payments': 4,
    'Low spent Medium value payments': 5,
    'High spent Large value payments': 6
}

# Asignar valor 7 a cualquier valor que no esté en el diccionario (Other)
df6_cleaned_final['Payment_Behaviour_Encoded'] = df6_cleaned_final['Payment_Behaviour'].map(payment_behaviour_mapping).fillna(7)

# Reemplazar columna original
df6_cleaned_final['Payment_Behaviour'] = df6_cleaned_final['Payment_Behaviour_Encoded']
df6_cleaned_final.drop(columns=['Payment_Behaviour_Encoded'], inplace=True)

# Codificación de columnas individuales de préstamos

loan_cols = [
    'Auto Loan', 'Credit-Builder Loan', 'Debt Consolidation Loan', 'Home Equity Loan',
    'Mortgage Loan', 'Not Specified', 'Payday Loan', 'Personal Loan', 'Student Loan'
]

# Mapeo Yes/No/Unknown → 2/1/0
loan_mapping = {'Unknown': 0, 'No': 1, 'Yes': 2}

for col in loan_cols:
    df6_cleaned_final[col + ' Encoded'] = df6_cleaned_final[col].map(loan_mapping)

# Codificar variable target Credit_Score
credit_score_mapping = {'Poor':0, 'Standard':1, 'Good':2}
df6_cleaned_final['Credit_Score_Encoded'] = df6_cleaned_final['Credit_Score'].map(credit_score_mapping)

# Eliminamos la columna original
df6_cleaned_final.drop(columns=['Credit_Score'], inplace=True)

#Verificamos codificación Credit_Score
print('Verificación codificación Credit_Score (y)')
print(df6_cleaned_final['Credit_Score_Encoded'].value_counts())

# Eliminar columnas de texto de préstamos
df6_cleaned_final.drop(columns=loan_cols, inplace=True)

# Verificación final

print("Dimensiones finales del dataset codificado:", df6_cleaned_final.shape)
print("Columnas codificadas (últimas):", df6_cleaned_final.columns[-10:].tolist())
print(df6_cleaned_final)
print(df6_cleaned_final['Credit_Score_Encoded'])
print(df6_cleaned_final['Credit_Mix_Encoded'])
print(df6_cleaned_final['Payment_of_Min_Amount_Encoded'])

Verificación codificación Occupation
        Occupation  Occupation_Encoded
0           Lawyer                   1
7         Mechanic                   2
14   Media_Manager                   3
22          Doctor                   4
37      Journalist                   5
44      Accountant                   6
51         Manager                   7
58    Entrepreneur                   8
66       Scientist                   9
88       Architect                  10
103        Teacher                  11
151       Engineer                  12
166         Writer                  13
203      Developer                  14
254       Musician                  15
Verificación codificación Credit_Score (y)
Credit_Score_Encoded
1    49182
0    26773
2    16445
Name: count, dtype: int64
Dimensiones finales del dataset codificado: (92400, 36)
Columnas codificadas (últimas): ['Auto Loan Encoded', 'Credit-Builder Loan Encoded', 'Debt Consolidation Loan Encoded', 'Home Equity Loan Encoded', 'Mortgage Lo

In [22]:
# Estandarización de variables numéricas

from sklearn.preprocessing import MinMaxScaler

# Columnas numéricas relevantes para similitud
cols_numericas = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
    'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
    'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance'
]

scaler = MinMaxScaler()
df6_cleaned_final[cols_numericas] = scaler.fit_transform(df6_cleaned_final[cols_numericas])

columnas_a_imprimir = [
    'Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 
    'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
    'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
    'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance'
]

#Verificación estandarización columnas numéricas
print(df6_cleaned_final[columnas_a_imprimir])

            Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0      0.980216       0.136547               0.161232           0.636364   
1      0.980216       0.136547               0.161232           0.636364   
2      0.980216       0.136547               0.161232           0.636364   
3      0.980216       0.136547               0.161232           0.636364   
4      0.980216       0.136547               0.161232           0.636364   
...         ...            ...                    ...                ...   
92395  0.951439       0.198822               0.231275           0.818182   
92396  0.951439       0.198822               0.231275           0.818182   
92397  0.951439       0.198822               0.231275           0.818182   
92398  0.951439       0.198822               0.231275           0.818182   
92399  0.951439       0.198822               0.231275           0.818182   

       Num_Credit_Card  Interest_Rate  Num_of_Loan  Delay_from_due_date  \
0           

In [23]:
#Diagnosticamos por última vez los valores nulos en las columnas del dataframe
for i in df6_cleaned_final.columns:
    print(i+':'+ str(df6_cleaned_final[i].isnull().sum()))

ID:0
Customer_ID:0
Month:0
Name:0
Age:0
SSN:0
Annual_Income:0
Monthly_Inhand_Salary:0
Num_Bank_Accounts:0
Num_Credit_Card:0
Interest_Rate:0
Num_of_Loan:0
Delay_from_due_date:0
Num_of_Delayed_Payment:0
Changed_Credit_Limit:0
Num_Credit_Inquiries:0
Outstanding_Debt:0
Credit_Utilization_Ratio:0
Credit_History_Age:0
Total_EMI_per_month:0
Amount_invested_monthly:0
Payment_Behaviour:0
Monthly_Balance:0
Occupation_Encoded:0
Credit_Mix_Encoded:0
Payment_of_Min_Amount_Encoded:0
Auto Loan Encoded:0
Credit-Builder Loan Encoded:0
Debt Consolidation Loan Encoded:0
Home Equity Loan Encoded:0
Mortgage Loan Encoded:0
Not Specified Encoded:0
Payday Loan Encoded:0
Personal Loan Encoded:0
Student Loan Encoded:0
Credit_Score_Encoded:0


In [24]:
print(sorted(df6_cleaned_final['Credit_Mix_Encoded'].dropna().unique()))
print(sorted(df6_clean['Credit_Mix'].dropna().unique()))

[0, 1, 2, 3]
['Bad', 'Good', 'Standard', 'Unknown']


In [25]:
# Detectar errores pre-exportación del csv

import re

# Copia de seguridad antes de limpiar
df_temp = df6_cleaned_final.copy()

# Patrón de caracteres problemáticos
patron_caracteres_invalidos = r'["\',]'

# Inicializar bandera
nombres_previos_invalidos = -1

while True:
    # Detectar nombres con caracteres indeseados
    errores_nombres = df_temp[df_temp['Name'].str.contains(patron_caracteres_invalidos, regex=True, na=False)]

    # Ver cuántos quedan
    errores_actuales = len(errores_nombres)

    print(f"[DEBUG] Nombres con caracteres problemáticos: {errores_actuales}")

    # Si no hay avance (mismo número de errores), salimos del bucle
    if errores_actuales == nombres_previos_invalidos:
        print("[INFO] No se detecta avance, se asume limpieza finalizada.")
        break

    # Actualizar bandera
    nombres_previos_invalidos = errores_actuales

    # Corrección de errores
    df_temp['Name'] = (
        df_temp['Name']
        .str.replace('"', '', regex=False)
        .str.replace("'", '', regex=False)
        .str.replace(r',+', ',', regex=True)  # múltiples comas → una sola
        .str.replace(',', '', regex=False)    # luego eliminar las comas
        .str.replace(' ', '', regex=False)    # eliminar espacios
        .str.strip()                          # limpiar bordes
    )

# Asignamos a df limpio final
df6_cleaned_final = df_temp.copy()

# Verificación final de los primeros valores únicos
print("\n[INFO] Primeros valores únicos de la columna Name post-limpieza:")
print(df6_cleaned_final['Name'].dropna().unique()[:30])

[DEBUG] Nombres con caracteres problemáticos: 882
[DEBUG] Nombres con caracteres problemáticos: 0
[DEBUG] Nombres con caracteres problemáticos: 0
[INFO] No se detecta avance, se asume limpieza finalizada.

[INFO] Primeros valores únicos de la columna Name post-limpieza:
['AlistairBarrf' 'Arunah' 'Shirboni' 'Schneyerh' 'Cameront' 'Holtono'
 'Felsenthalq' 'Josephv' 'NeilChatterjeex' 'Rhysn' 'Wahbap' 'Matthewm'
 'B.h' 'TimKellyf' 'JonathanStempela' 'MakiShirakip' 'JanetMcGurtyg'
 'Leahk' 'Kentaros' 'Markm' 'LuciaMutikanin' 'Hornbyb' 'DavidSheppardv'
 'PatriciaZengerlel' 'Lopezz' 'SylviaWestallq' 'AndreasCremero'
 'AlwynScottk' 'Dorfmanv' 'Karenh']


In [26]:
df6_cleaned_final.to_csv('train_clean(codificado).csv', index=False)

import os
print(os.getcwd())

d:\Ciencia de Datos y Análisis con Python\Bases de Datos Kaggle\Credit Score Classification feb\Archivos Repositorio Github


In [27]:
print(df6_cleaned_final.dtypes)

ID                                  object
Customer_ID                         object
Month                               object
Name                                object
Age                                float64
SSN                                 object
Annual_Income                      float64
Monthly_Inhand_Salary              float64
Num_Bank_Accounts                  float64
Num_Credit_Card                    float64
Interest_Rate                      float64
Num_of_Loan                        float64
Delay_from_due_date                float64
Num_of_Delayed_Payment             float64
Changed_Credit_Limit               float64
Num_Credit_Inquiries               float64
Outstanding_Debt                   float64
Credit_Utilization_Ratio           float64
Credit_History_Age                 float64
Total_EMI_per_month                float64
Amount_invested_monthly            float64
Payment_Behaviour                    int64
Monthly_Balance                    float64
Occupation_

In [28]:
print(df6_clean.dtypes)

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                         float64
SSN                          object
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                 float64
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age          float64
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly     float64
Payment_Behaviour            object
Monthly_Balance             float64
Credit_Score                 object
Payday Loan                 